In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image
from IPython.display import display

In [2]:
def laplace(u,i,j):
    sumA = 0
    sumA += u[i][j] * -20    
    sumA += u[i-1][j] * 4
    sumA += u[i+1][j] * 4
    sumA += u[i][j-1] * 4
    sumA += u[i][j+1] * 4    
    sumA += u[i-1][j-1] 
    sumA += u[i-1][j+1] 
    sumA += u[i+1][j-1] 
    sumA += u[i+1][j+1]    
    return sumA / 20

In [3]:
def turing(shape = 100, dot_length = 5, dot_width = 5,  Da = 1, Db = 0.5, feed = 0.028, kill = 0.062, 
           spot = 'random', color = False, color_inverse = False, dt = 1,  RGB = (190, 125, 100), 
           color_spot = "R", step = 50, stop = 30000):
    
    if type(shape) == int: 
        if shape < 60:
            print("shape should be greater than 60, ")
            shape = 60
        else:
            shape = (shape, shape)

            
    img = Image.new('RGB', (shape[0], shape[1]), color = (255,255,255))

    #set initial conditions
    U = np.ones((shape[0], shape[1]))
    V = np.zeros((shape[0], shape[1]))
    initial_matrices = (np.copy(U) , np.copy(V))
    U,V = np.copy(initial_matrices[0]), np.copy(initial_matrices[1])
    
    #creating disturbance
    if spot == 'random':
        for q in range(shape[0]//20):
            x_start = random.randint(1, shape[0] - 2*dot_length - 1)
            y_start = random.randint(1, shape[1] - 2*dot_width - 1)
            U[x_start : x_start + 2* dot_length , y_start: y_start+ 2* dot_width] = 0
            V[x_start : x_start + 2* dot_length , y_start: y_start + 2* dot_width] = 0.5  
           
    elif spot == 'two vertical':
        #up dot
        U[(shape[0]//2)-dot_length :(shape[0]//2)+dot_length , (shape[1]//2)-dot_width - 2*dot_width:(shape[1]//2)+dot_width - 2*dot_width] = 0
        V[shape[0]//2-dot_length :shape[0]//2+dot_length , shape[1]//2-dot_width - 2*dot_width:shape[1]//2+dot_width - 2*dot_width] = 0.5
        #down dot
        U[(shape[0]//2)-dot_length :(shape[0]//2)+dot_length  , (shape[1]//2)-dot_width + 2*dot_width:(shape[1]//2)+dot_width + 2*dot_width] = 0
        V[shape[0]//2-dot_length :shape[0]//2+dot_length  , shape[1]//2-dot_width + 2*dot_width:shape[1]//2+dot_width + 2*dot_width] = 0.5
    elif spot == 'two horizontal':
        #left dot
        U[(shape[0]//2)-dot_length - 2*dot_length:(shape[0]//2)+dot_length - 2*dot_length , (shape[1]//2)-dot_width:(shape[1]//2)+dot_width] = 0
        V[shape[0]//2-dot_length - 2*dot_length:shape[0]//2+dot_length - 2*dot_length , shape[1]//2-dot_width:shape[1]//2+dot_width] = 0.5
        #right dot
        U[(shape[0]//2)-dot_length + 2*dot_length:(shape[0]//2)+dot_length + 2*dot_length , (shape[1]//2)-dot_width:(shape[1]//2)+dot_width] = 0
        V[shape[0]//2-dot_length + 2*dot_length:shape[0]//2+dot_length + 2*dot_length , shape[1]//2-dot_width:shape[1]//2+dot_width] = 0.5
    elif spot == 'diagonal':
        #left dot
        U[(shape[0]//2)-dot_length - 2*dot_length:(shape[0]//2)+dot_length - 2*dot_length , (shape[1]//2)-dot_width - 2*dot_width:(shape[1]//2)+dot_width - 2*dot_width] = 0
        V[shape[0]//2-dot_length - 2*dot_length:shape[0]//2+dot_length - 2*dot_length , shape[1]//2-dot_width - 2*dot_width :shape[1]//2+dot_width - 2*dot_width] = 0.5
        #right dot
        U[(shape[0]//2)-dot_length + 2*dot_length:(shape[0]//2)+dot_length + 2*dot_length , (shape[1]//2)-dot_width+ 2*dot_width:(shape[1]//2)+dot_width + 2*dot_width] = 0
        V[shape[0]//2-dot_length + 2*dot_length:shape[0]//2+dot_length + 2*dot_length , shape[1]//2-dot_width + 2*dot_width :shape[1]//2+dot_width + 2*dot_width] = 0.5
    else:
         # making spot in the middle
         U[(shape[0]//2)-dot_length :(shape[0]//2)+dot_length , (shape[1]//2)-dot_width:(shape[1]//2)+dot_width] = 0
         V[shape[0]//2-dot_length:shape[0]//2+dot_length , shape[1]//2-dot_width:shape[1]//2+dot_width] = 0.5        
       
    for o in range(stop):        
        Unext = np.ones((shape[0], shape[1]))
        Vnext = np.zeros((shape[0], shape[1]))
        
        for i in range(1, shape[0]-1):
            for j in range(1, shape[1]-1):
                u,v = U[i][j], V[i][j]                 
                uNext = u + ((Da * laplace(U,i,j)) - (u*v*v) + (feed*(1 - u))) * dt
                vNext = v + ((Db * laplace(V,i,j)) + (u*v*v) - ((feed + kill) * v)) * dt                
                uNext = max(min(1, uNext), 0)
                vNext = max(min(1, vNext), 0)
                Unext[i][j], Vnext[i][j] = uNext, vNext
                
                if color == False:
                    dif_uv = uNext - vNext
                    dif_uv = max(dif_uv, 0)
                    dif_uv = int(dif_uv * 255)
                    if color_inverse:
                        diff_uv = 255-dif_uv
                    img.putpixel((i, j), (dif_uv, dif_uv, dif_uv, 255) )
                else:
                    uN = uNext
                    if color_inverse:
                        uN = 1 - uNext
                    if color_spot == 'B':
                        img.putpixel((i, j), (int(RGB[0]*uN), int(RGB[1]*uN), int(RGB[2]*(1-uN)), 255) )                        
                    elif color_spot == 'G':
                        img.putpixel((i, j), (int(RGB[0]*uN), int(RGB[1]*(1-uN)), int(RGB[2]*uN), 255) )
                    else:
                        img.putpixel((i, j), (int(RGB[0]*(1-uN)), int(RGB[1]*uN), int(RGB[2]*uN), 255) )
                    
                
        U,V = np.copy(Unext), np.copy(Vnext)
        
        if o % step == 0:
            print(o)
            #picture = img.save("Images/" +  str(feed) + "_"+ str(kill)+ "_" + str(o)   +".jpg") 
            display(img)
    return None

In [ ]:
#Spot. Example of mitosis
turing(shape = 200, dot_length= 8, dot_width= 4, feed = 0.028, kill = 0.062, spot = "center", stop = 7500)

In [ ]:
#Labyrinthine stripes. Example of a Maze
turing(shape = 200, dot_length= 8, dot_width=8, feed = 0.29, kill = 0.57, spot = "random",
       color = True,  RGB = (120, 170, 100), color_spot = "R", color_inverse = True, stop = 7000)

In [ ]:
#more spots fewer stripes
turing(shape = (200, 200), dot_length= 3, dot_width= 3, feed = 0.0315, kill = 0.061, 
       spot = "random", step = 100, stop = 24000)

In [ ]:
#mixture. comparative spots and stripes
turing(shape = (200, 200), dot_length= 3, dot_width= 3, feed = 0.0321, kill = 0.061, 
       spot = "random", step = 150, stop = 24000)

In [ ]:
#more stripes fewer spots
turing(shape = (200, 200), dot_length= 3, dot_width= 3, feed = 0.033, kill = 0.061, 
       spot = "random", step = 100, stop 12500)

In [ ]:
#Tiger's tail
def turing(shape = (200, 50), dot_length = 3, dot_width = 3,  Da = 1, Db = 0.5, feed = 0.038, kill = 0.061, spot = 'random', dt = 1,  RGB = (190, 125, 100), step = 200, stop = 30000  ):
       
    img = Image.new('RGB', (shape[0], shape[1]), color = (255,255,255))
    
    #set initial conditions
    U = np.ones((shape[0], shape[1]))
    V = np.zeros((shape[0], shape[1]))
    initial_matrices = (np.copy(U) , np.copy(V))
    U,V = np.copy(initial_matrices[0]), np.copy(initial_matrices[1])
    
    #creating disturbance
    if spot == 'random':
        for q in range(shape[0]//20):
            x_start = random.randint(1, shape[0] - 2*dot_length - 1)
            y_start = random.randint(1, shape[1] - 2*dot_width - 1)
            U[x_start : x_start + 2* dot_length , y_start: y_start+ 2* dot_width] = 0
            V[x_start : x_start + 2* dot_length , y_start: y_start + 2* dot_width] = 0.5  
           
    for o in range(stop):        
        Unext = np.ones((shape[0], shape[1]))
        Vnext = np.zeros((shape[0], shape[1]))
        
        for i in range(1, shape[0]-1):
            for j in range(1, shape[1]-1):
                u,v = U[i][j], V[i][j]                 
                uNext = u + ((Da * laplace(U,i,j)) - (u*v*v) + (feed*(1 - u))) * dt
                vNext = v + ((Db * laplace(V,i,j)) + (u*v*v) - ((feed + kill) * v)) * dt
                
                uNext = max(min(1, uNext), 0)
                vNext = max(min(1, vNext), 0)
                Unext[i][j], Vnext[i][j] = uNext, vNext
                
                dif_uv = uNext - vNext
                dif_uv = max(dif_uv, 0)
                dif_uv = int(dif_uv * 255) 
                if dif_uv > 160:
                    img.putpixel((i, j), (255, 255, 0, 255) )
                else: 
                    img.putpixel((i,j), (dif_uv, dif_uv, dif_uv))
                
                
                    
                
        U,V = np.copy(Unext), np.copy(Vnext)
        
        if o % step == 0:
            print(o)
            #save pictures
            #picture = img.save("Images/" +  str(feed) + "_"+ str(kill)+ "_" + str(o)   +".jpg") 
            display(img)
    return None
turing()